# ChiffonCore bring-up (Ultra96V2 / PYNQ)

1. Load bitstream (Overlay)
2. Write `.hex` to DRAM (CMA buffer)
3. Set `dram_base` / `entry_pc` and `START` signal via regbus


In [ ]:
from pathlib import Path
import numpy as np
from pynq import Overlay, allocate


## 1) Load Overlay


In [ ]:
BIT_PATH = "design_1.bit"
ol = Overlay(BIT_PATH)
ol


## 2) Get regbus handle


In [ ]:
# Find regbus
cands = [k for k in ol.ip_dict.keys() if "regbus" in k.lower()]
print("regbus candidates:", cands)
assert len(cands) > 0, "regbus IP not found. HINT: check ol.ip_dict."
regbus = getattr(ol, cands[0])
regbus


## 2.5) Configure DisplayPort output (required)

Ultra96-V2 / PYNQ の DisplayPort 出力を有効化します。
`pynq_draw_patblt_test.ipynb` と同じ `LiveDisplayPort` を使います。


In [ ]:
import time
from pynq import MMIO
from pynq.lib.video import *

class LiveDisplayPort(DisplayPort):
    
    # DisplayPortレジスタパラメータ
    __DP_REG_BASE_ADDR__                    = 0xfd4a0000
    __DP_REG_ADDR_RANGE__                   = 0x0000cc20
    __RA_DP_MAIN_STREAM_ENABLE__            = 0x00000084
    __RA_V_BLEND_SET_GLOVAL_ALPHA_REG__     = 0x0000a00c
    __RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__ = 0x0000b070
    __RA_AV_BUF_AUD_VID_CLK_SOURCE__        = 0x0000b120
    __RA_AV_BUF_SRST_REG__                  = 0x0000b124
    
    def __init__(self, event_loop=None):
        super().__init__(event_loop)
        
    def configure(self, mode, pixelformat):
        super().configure(mode, pixelformat)
        self.__setup_live_video__()
        
    def __setup_live_video__(self):
        dpreg = MMIO(self.__DP_REG_BASE_ADDR__, self.__DP_REG_ADDR_RANGE__)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000000)
        dpreg.write(self.__RA_V_BLEND_SET_GLOVAL_ALPHA_REG__, 0x00000000)
        dpreg.write(self.__RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__, 0x00000050)
        dpreg.write(self.__RA_AV_BUF_AUD_VID_CLK_SOURCE__, 0x00000002)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000002)
        time.sleep(1)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000000)
        time.sleep(1)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000001)

In [ ]:
from pynq.lib.video import *
lvdp = LiveDisplayPort()

PIXEL_WIDTH  = PIXEL_VGA_WIDTH
PIXEL_HEIGHT = PIXEL_VGA_HEIGHT
print("VGA setting @ DisplayPort", PIXEL_WIDTH, PIXEL_HEIGHT)

lvdp.configure(VideoMode(PIXEL_WIDTH, PIXEL_HEIGHT, 24), PIXEL_RGB)

## 3) Define bootctrl

- `STATUS`  : `BOOT_BASE + 0x0`  (bit1=run, bit0=hold_reset)
- `CTRL`    : `BOOT_BASE + 0x4`  (bit0=hold_reset level, bit1=START W1P)
- `DRAMBASE`: `BOOT_BASE + 0x8`
- `ENTRYPC` : `BOOT_BASE + 0xC`


In [ ]:
BOOT_BASE = 0x1000

RA_STATUS   = BOOT_BASE + 0x0000
RA_CTRL     = BOOT_BASE + 0x0004
RA_DRAMBASE = BOOT_BASE + 0x0008
RA_ENTRYPC  = BOOT_BASE + 0x000C


## 4) Load .hex into DDR RAM + allocate framebuffer + write mailbox (for CPU->regbus draw/display test)

This notebook will:
- allocate a DDR RAM buffer (CPU RAM backing store)
- load a program .hex into the beginning of that buffer
- allocate a framebuffer `fb0` in DDR (CMA)
- write `fb0.device_address` into a mailbox location that the CPU program reads (default: CPU addr 0x8000_2000)

Assumptions:
- CPU RAM base (virtual) is 0x8000_0000 and is translated to DDR via (cpu_addr - 0x8000_0000) + ENTRYPC.
- regbus MMIO base is already wired so the CPU program can command draw/display.


In [ ]:
# --- Program to run ---
# Put your program hex here:
HEX_PATH = "cpu_draw_display_min_test.hex"   # <-- change if needed
BYTESWAP32 = False

# --- CPU RAM mapping (virtual) ---
CPU_RAM_BASE      = 0x8000_0000
MAILBOX_CPU_ADDR  = 0x8000_2000  # CPU reads framebuffer phys addr from here (uint64)
assert (MAILBOX_CPU_ADDR % 8) == 0

# --- Framebuffer (same as draw notebook default VGA) ---
PIXEL_WIDTH  = 640
PIXEL_HEIGHT = 480
PIXEL_CH     = 4  # RGBA bytes

# --- RAM size to allocate for CPU ---
# For small tests you only need a bit more than program size + mailbox offset.
# For larger programs / riscv-tests / OS, increase this (e.g., 16-64 MiB).
RAM_BYTES = 16 * 1024 * 1024  # 16 MiB


def parse_hex_words(path: str):
    words = []
    for line in Path(path).read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or line.startswith("//"):
            continue
        if line.startswith("0x") or line.startswith("0X"):
            line = line[2:]
        v = int(line, 16) & 0xFFFF_FFFF
        if BYTESWAP32:
            v = int.from_bytes(v.to_bytes(4, "big"), "little")
        words.append(v)
    return words


# 1) Allocate framebuffer in DDR (CMA)
fb0 = allocate(shape=(PIXEL_HEIGHT, PIXEL_WIDTH, PIXEL_CH), dtype=np.uint8, cacheable=False)
fb0[:] = 0
fb0.flush()
fb_phys = int(fb0.device_address)
print(f"fb0.device_address = 0x{fb_phys:08x}  ({PIXEL_WIDTH}x{PIXEL_HEIGHT}x{PIXEL_CH} bytes)")

assert fb_phys <= 0xFFFF_FFFF, "AXI_ADDR_WIDTH=32 なので 4GB を超える framebuffer 物理アドレスは使えません。"


# 2) Load program hex into DDR RAM buffer (CPU RAM backing store)
words32 = parse_hex_words(HEX_PATH)
print("32-bit words:", len(words32))

prog_qwords = (len(words32) + 1) // 2

mailbox_off_bytes = MAILBOX_CPU_ADDR - CPU_RAM_BASE
assert mailbox_off_bytes >= 0
mailbox_qi = mailbox_off_bytes // 8

ram_qwords = max((RAM_BYTES + 7) // 8, prog_qwords, mailbox_qi + 1)

buf = allocate(shape=(ram_qwords,), dtype=np.uint64, cacheable=False)
buf[:] = 0

for qi in range(prog_qwords):
    lo = words32[2*qi] if (2*qi) < len(words32) else 0
    hi = words32[2*qi + 1] if (2*qi + 1) < len(words32) else 0
    buf[qi] = np.uint64((hi << 32) | lo)

# 3) Write mailbox: framebuffer physical address (uint64)
buf[mailbox_qi] = np.uint64(fb_phys)

buf.flush()

entry_pc = int(buf.device_address)  # DDR physical address used by bootctrl
print(f"CPU RAM buf.device_address (ENTRYPC) = 0x{entry_pc:08x}  (RAM_BYTES={RAM_BYTES} bytes)")
print(f"mailbox: cpu=0x{MAILBOX_CPU_ADDR:08x} -> buf qword idx={mailbox_qi} -> phys=0x{entry_pc + mailbox_qi*8:08x}")

assert entry_pc <= 0xFFFF_FFFF, "AXI_ADDR_WIDTH=32 なので 4GB を超える ENTRYPC は使えません。"


## 5) Write settings to bootctrl and START

`CTRL = 0b10` を 1回書くと：
- hold_reset = 0（解除）
- run = 1（START W1P）


In [ ]:
regbus.write(RA_CTRL, 0x1)  # hold_reset=1

# base and entry
regbus.write(RA_DRAMBASE, 0x0000_0000)
regbus.write(RA_ENTRYPC,  entry_pc & 0xFFFF_FFFF)

# START (release reset + run)
regbus.write(RA_CTRL, 0x2)

st = regbus.read(RA_STATUS)
print(f"STATUS=0x{st:08x}  (run={(st>>1)&1}, hold_reset={st&1})")


## 6) What to expect

If your CPU program is `cpu_draw_display_min_test.hex`:
- the CPU reads the framebuffer physical address from the mailbox (0x8000_2000)
- then writes draw/display commands via regbus MMIO
- you should see the VGA/HDMI output fill with a solid color (e.g., red) within a short time.

If nothing changes:
- first confirm the mailbox index/phys print above looks sane
- then confirm your regbus MMIO decode routes the CPU writes to draw/display
- and confirm your store path generates correct write data + byte-lanes for regbus.
